## Q2 LOGISTIC REGRESSION

## Importing Libraries

In [0]:
from PIL import Image
import numpy as np
import sys
import os
import csv
import pandas as pd
from scipy.linalg import eigh

# Generating name of files

In [0]:
def FileList(myDir, format='.jpg'):
    fileList = []
#     print(myDir)
    ext=".jpg"
    for root,dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(name,ext)
                fileList.append(name)
    return fileList

In [0]:
filelist=FileList("/content/drive/My Drive/Dataset/")
filelist.sort()
print(filelist)

['000_000.jpg', '000_001.jpg', '000_002.jpg', '000_003.jpg', '000_004.jpg', '000_005.jpg', '000_006.jpg', '000_007.jpg', '000_008.jpg', '000_009.jpg', '000_010.jpg', '000_011.jpg', '000_012.jpg', '000_013.jpg', '000_014.jpg', '000_015.jpg', '000_016.jpg', '000_017.jpg', '000_018.jpg', '000_019.jpg', '000_020.jpg', '000_021.jpg', '000_022.jpg', '000_023.jpg', '000_024.jpg', '000_025.jpg', '000_026.jpg', '000_027.jpg', '000_028.jpg', '000_029.jpg', '000_030.jpg', '000_031.jpg', '000_032.jpg', '000_033.jpg', '000_034.jpg', '000_035.jpg', '000_036.jpg', '000_037.jpg', '000_038.jpg', '000_039.jpg', '000_040.jpg', '000_041.jpg', '000_042.jpg', '000_043.jpg', '000_044.jpg', '000_045.jpg', '000_046.jpg', '000_047.jpg', '000_048.jpg', '000_049.jpg', '000_050.jpg', '000_051.jpg', '000_052.jpg', '000_053.jpg', '000_054.jpg', '000_055.jpg', '000_056.jpg', '000_057.jpg', '000_058.jpg', '000_059.jpg', '000_060.jpg', '000_061.jpg', '000_062.jpg', '000_063.jpg', '000_064.jpg', '001_000.jpg', '001_001.

# Making Of DataFrame

In [0]:
imageset=[]
for file in filelist:
    im1= Image.open("/content/drive/My Drive/Dataset/"+file)
    newsize = (64,64) 
    im1 = im1.resize(newsize) 
#     # Make image Greyscale
    im1 = im1.convert('L')
#     # Save Greyscale values
    value = np.asarray(im1.getdata(), dtype=np.int)
    value = value.flatten()
    imageset.append(value)


# Feature Scaling

In [0]:
from sklearn.preprocessing import StandardScaler
imageset=pd.DataFrame(imageset)
# feature scaling
scaler = StandardScaler()
imageset=scaler.fit_transform(imageset)

In [0]:
print(imageset)

[[ 1.40233900e+01  2.26536097e+01 -1.21611789e+00 ... -5.18312513e-02
  -2.85428852e-01  2.78258206e-01]
 [-1.51477303e+01  1.51782985e+01 -3.89438046e+00 ... -1.31425029e-02
   2.77448230e-01  5.30562596e-02]
 [ 5.32835179e+00 -1.62042300e+01  1.05197750e+01 ...  8.31007432e-01
  -5.29106689e-02 -5.54865330e-01]
 ...
 [-1.56864569e+01 -4.93055853e+00  1.37229427e+01 ...  2.12371123e-01
  -4.48827290e-01  3.69299647e-01]
 [ 9.37565772e+00 -3.42285526e+01 -1.26439816e+00 ... -1.48577164e-01
   4.29235081e-01 -1.82311956e-01]
 [ 6.52242096e+00 -4.37569852e+01  8.85894793e-01 ...  2.09911328e-01
  -3.57699262e-01 -2.78176179e-02]]


# PCA Function

In [0]:
def PCA(A,K):
    covarMat=np.cov(A.T)
    values, vectors=eigh(covarMat)
    idx= np.argsort(values)
    idx = idx[::-1]
    eigvec = vectors[:,idx]
    eigval = values[idx]
    subeigvec=eigvec[:,:K]
    newdata=np.dot(A,subeigvec)
    return newdata

# Applying PCA and taking 400 Components

In [0]:
# print(imageset.shape)
imageset=PCA(imageset,400)
print(imageset.shape)

(520, 400)


In [0]:
from sklearn.model_selection import train_test_split
df=np.array(imageset)
li=[1.0]*(df.shape[0])
li=np.array(li)
li.shape=(520,1)
df= np.hstack((li,df))


# Splitting DataFrame into train and test data

In [0]:
# construction of test data
actual_labels=[]
for i in range(0,8):
    for j in range(0,65):
        actual_labels.append(i)
actual_labels=np.array(actual_labels)
#splitting of data
X_train, X_test, y_train, y_test = train_test_split(df,actual_labels, test_size=0.2)

In [0]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(416, 401)
(104, 401)
(416, 1)
(104,)


# LOGISTIC REGRESSION

In [0]:
def logisticfun(z):
  return 1.0 / (1 + np.exp(-z))

# Training Model

In [0]:
def fit(parameters,X_train,y_train):
  alpha=0.02
  for j in range(0,1000):
    z=np.dot(X_train,parameters)
    pred_list=logisticfun(z)
    predlist=np.asarray(pred_list)
    predlist=np.subtract(predlist,y_train)
    df1=X_train.transpose()
    intermediate= np.dot(df1,predlist)
    # print(intermediate.shape)
    for i in range(0,401):
      intermediate[i]=intermediate[i]*(1/y_train.shape[0])
    intermediate*=alpha
    parameters=np.subtract(parameters,intermediate)
  # print(parameters)
  return parameters

# Initializing Parameters

In [0]:
parameters = np.zeros(X_train.shape[1])
parameters=np.expand_dims(parameters, axis=1)
parameters.shape

(401, 1)

# MultiClass Classification in Logistic Regression

In [0]:
Rparameters=[]
temp=[]
for i in range(0,8):
  temp=y_train
  for j in range(0,X_train.shape[0]):
    if(y_train[j]==i):
      temp[j]=1
    else:
      temp[j]=0
  temp.shape=(y_train.shape[0],1)
  Rparameters.append(fit(parameters,X_train,temp))
Rparameters[0].shape
# print(Rparameters[1])

(401, 1)

# Prediction Function

In [0]:
def predict_prob(X_test,Rparameters):
  problist=[]
  for k in range(0,X_test.shape[0]):
    temp=[]
    for i in range(0,8):
      temp.append(logisticfun(np.dot(X_test[k,:],Rparameters[i])))
    # print(temp)
    maxpos = temp.index(max(temp))
    # print(maxpos)
    problist.append(maxpos)
  return problist

In [0]:
t_predlist=predict_prob(X_test,Rparameters)

In [0]:
t_pred=np.array(t_predlist)
t_pred.shape=(y_test.shape[0],1)

# Accuracy Score

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, t_pred)

0.5442307692307692
